In [1]:
# Standard library imports
from nltk import pos_tag
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

# Third-party imports
import numpy as np
import pandas as pd
import tensorflow as tf
from scikeras.wrappers import KerasClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_validate, train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from tensorflow.keras import Sequential, layers

# Global constants
STOPWORDS = stopwords.words('english')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

df = pd.read_csv('../data/Airline_review.csv')[['Review_Title','Review','Recommended']]
reviews = df['Review_Title'] + ' ' + df['Review']
labels = df['Recommended'].map({'yes':1,'no':0})
train_reviews, temp_reviews, train_labels, temp_labels = train_test_split(reviews, labels, test_size=0.2, stratify=labels, random_state=42)
val_reviews, test_reviews, val_labels, test_labels = train_test_split(temp_reviews, temp_labels, test_size=0.5, stratify=temp_labels, random_state=42)

# Concatenating the valildation set as I don't need it here. 90-10 split
X_train = pd.concat([train_reviews, val_reviews])
y_train = pd.concat([train_labels, val_labels])

2024-03-29 13:38:30.061835: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
class TextCleanerTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, stop_words=None, lemmatize=True):
        self.stop_words = stop_words
        self.lemmatize = lemmatize
        
    def fit(self, X, y=None):
        return self 
    
    def transform(self, X, y=None):
        cleaned_reviews = []
        for review in X:
            cleaned_reviews.append(self.clean_text(review, self.stop_words, self.lemmatize))
        return cleaned_reviews
    
    def clean_text(self, review, stop_words, lemmatize):
        tokenizer = RegexpTokenizer(r"([a-zA-Z]+(?:’[a-z]+)?)")
        tokens = tokenizer.tokenize(review)
        if stop_words is None:
            tokens = [word.lower() for word in tokens]
        else:
            tokens = [word.lower() for word in tokens if word.lower() not in stop_words]

        if lemmatize:
            pos_tags = pos_tag(tokens)
            wordnet_tags = [(word, self.get_wordnet_pos(tag)) for word, tag in pos_tags]
            lemmatizer = WordNetLemmatizer()
            lemmatized_tokens = [lemmatizer.lemmatize(word, tag) for word, tag in wordnet_tags]
            return ' '.join(lemmatized_tokens)
        else:
            return ' '.join(tokens)
    
    def get_wordnet_pos(self, treebank_tag):
        if treebank_tag.startswith('J'):
            return wordnet.ADJ
        elif treebank_tag.startswith('V'):
            return wordnet.VERB
        elif treebank_tag.startswith('N'):
            return wordnet.NOUN
        elif treebank_tag.startswith('R'):
            return wordnet.ADV
        else:
            return wordnet.NOUN
STOPWORDS = stopwords.words('english')
text_cleaner = TextCleanerTransformer(stop_words=STOPWORDS, lemmatize=True)
pipe = Pipeline([('cleaner',text_cleaner),
                 ('vectorizer', CountVectorizer(decode_error='replace', strip_accents='unicode', stop_words=None, ngram_range=(1, 2), max_df=0.95, min_df=2)),
                 ('tf_idf', TfidfTransformer()),
                 ('feature_selection', SelectKBest(k=10000))])
X_clean = pipe.fit_transform(X_train, y_train)

In [17]:
def build_mlp_model(num_layers=2, units=64):
    model = Sequential()
    model.add(layers.InputLayer(input_shape=(10000,)))
    for _ in range(num_layers - 1):
        model.add(layers.Dense(units, activation="relu"))
        units = units // 2 
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

model_wrapper = KerasClassifier(build_fn=build_mlp_model,
                                random_state=42,
                                optimizer='adam',
                                loss='binary_crossentropy',
                                metrics=['accuracy'],
                                batch_size=64,
                                verbose=1,
                                callbacks=None,
                                shuffle=True,
                                epochs=2)



In [18]:
# just Gridserching the model
params = {
    'model__num_layers': [2, 3],  
    'model__units': [32, 64]}

gs = GridSearchCV(estimator=model_wrapper, 
                  param_grid=params,
                  scoring='accuracy',
                  cv=2,
                  verbose=3,
                  error_score='raise')

# The fitting process would be initiated with actual data
grid_search = gs.fit(X_clean, y_train)

Fitting 2 folds for each of 4 candidates, totalling 8 fits
Epoch 1/2


/Users/ronlodetti/anaconda3/envs/capstone2-env/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


163/163 [==============================] - 1s 4ms/step - loss: 0.5144 - accuracy: 0.8204
Epoch 2/2
163/163 [==============================] - 0s 2ms/step
[CV 1/2] END model__num_layers=2, model__units=32;, score=0.907 total time=   3.3s
Epoch 1/2


/Users/ronlodetti/anaconda3/envs/capstone2-env/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


163/163 [==============================] - 1s 4ms/step - loss: 0.5170 - accuracy: 0.8206
Epoch 2/2
163/163 [==============================] - 0s 2ms/step
[CV 2/2] END model__num_layers=2, model__units=32;, score=0.908 total time=   2.9s
Epoch 1/2


...

163/163 [==============================] - 2s 6ms/step - loss: 0.3767 - accuracy: 0.8290
Epoch 2/2
163/163 [==============================] - 0s 2ms/step
[CV 1/2] END model__num_layers=3, model__units=64;, score=0.916 total time=   4.2s
Epoch 1/2


/Users/ronlodetti/anaconda3/envs/capstone2-env/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


163/163 [==============================] - 2s 6ms/step - loss: 0.3817 - accuracy: 0.8328
Epoch 2/2
163/163 [==============================] - 0s 2ms/step
[CV 2/2] END model__num_layers=3, model__units=64;, score=0.920 total time=   4.1s
Epoch 1/2


/Users/ronlodetti/anaconda3/envs/capstone2-env/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


326/326 [==============================] - 3s 6ms/step - loss: 0.2927 - accuracy: 0.8733
Epoch 2/2
326/326 [==============================] - 2s 6ms/step - loss: 0.1459 - accuracy: 0.9456


In [20]:
# Now within a basic pipe
pipe = Pipeline([
    ('mlp', model_wrapper)
])

params = {
    'mlp__model__num_layers': [2, 3],  
    'mlp__model__units': [32, 64]}

gs = GridSearchCV(estimator=pipe, 
                  param_grid=params,
                  scoring='accuracy',
                  cv=2,
                  verbose=3,
                  error_score='raise')

# The fitting process would be initiated with actual data
grid_search = gs.fit(X_clean, y_train)

Fitting 2 folds for each of 4 candidates, totalling 8 fits
Epoch 1/2


/Users/ronlodetti/anaconda3/envs/capstone2-env/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


163/163 [==============================] - 1s 4ms/step - loss: 0.5144 - accuracy: 0.8204
Epoch 2/2
163/163 [==============================] - 0s 2ms/step
[CV 1/2] END mlp__model__num_layers=2, mlp__model__units=32;, score=0.907 total time=   3.2s
Epoch 1/2


/Users/ronlodetti/anaconda3/envs/capstone2-env/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


163/163 [==============================] - 1s 3ms/step - loss: 0.5170 - accuracy: 0.8206
Epoch 2/2
163/163 [==============================] - 0s 2ms/step
[CV 2/2] END mlp__model__num_layers=2, mlp__model__units=32;, score=0.908 total time=   2.8s
Epoch 1/2


...

163/163 [==============================] - 2s 6ms/step - loss: 0.3767 - accuracy: 0.8290
Epoch 2/2
163/163 [==============================] - 0s 2ms/step
[CV 1/2] END mlp__model__num_layers=3, mlp__model__units=64;, score=0.916 total time=   3.8s
Epoch 1/2


/Users/ronlodetti/anaconda3/envs/capstone2-env/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


163/163 [==============================] - 2s 6ms/step - loss: 0.3817 - accuracy: 0.8328
Epoch 2/2
163/163 [==============================] - 0s 2ms/step
[CV 2/2] END mlp__model__num_layers=3, mlp__model__units=64;, score=0.920 total time=   4.0s
Epoch 1/2


/Users/ronlodetti/anaconda3/envs/capstone2-env/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


326/326 [==============================] - 3s 6ms/step - loss: 0.2927 - accuracy: 0.8733
Epoch 2/2
326/326 [==============================] - 2s 6ms/step - loss: 0.1459 - accuracy: 0.9456


In [21]:
text_cleaner = TextCleanerTransformer(stop_words=STOPWORDS, lemmatize=True)
X_train_clean = text_cleaner.transform(X_train)

In [22]:
# Full pipe!
vectorizer = CountVectorizer(decode_error='replace', strip_accents='unicode', stop_words=None, ngram_range=(1, 2), max_df=0.95, min_df=2)
tf_idf = TfidfTransformer()
k_best = SelectKBest(k=10000)

pipe = Pipeline([
    ("count", vectorizer),
    ('tf_idf', tf_idf),
    ('feature_selection', k_best),
    ('mlp', model_wrapper)
])

params = {
    'mlp__model__num_layers': [2, 3],  
    'mlp__model__units': [32, 64]}

gs = GridSearchCV(estimator=pipe, 
                  param_grid=params,
                  scoring='accuracy',
                  cv=2,
                  verbose=3,
                  error_score='raise')

# The fitting process would be initiated with actual data
grid_search = gs.fit(X_train_clean, y_train)

Fitting 2 folds for each of 4 candidates, totalling 8 fits
Epoch 1/2


/Users/ronlodetti/anaconda3/envs/capstone2-env/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


163/163 [==============================] - 1s 4ms/step - loss: 0.5068 - accuracy: 0.8326
Epoch 2/2
163/163 [==============================] - 0s 2ms/step
[CV 1/2] END mlp__model__num_layers=2, mlp__model__units=32;, score=0.902 total time=   5.9s
Epoch 1/2


/Users/ronlodetti/anaconda3/envs/capstone2-env/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


163/163 [==============================] - 2s 4ms/step - loss: 0.5104 - accuracy: 0.8258
Epoch 2/2
163/163 [==============================] - 0s 2ms/step
[CV 2/2] END mlp__model__num_layers=2, mlp__model__units=32;, score=0.902 total time=   6.2s
Epoch 1/2


...

163/163 [==============================] - 2s 8ms/step - loss: 0.3654 - accuracy: 0.8404
Epoch 2/2
163/163 [==============================] - 0s 2ms/step
[CV 1/2] END mlp__model__num_layers=3, mlp__model__units=64;, score=0.904 total time=   9.3s
Epoch 1/2


/Users/ronlodetti/anaconda3/envs/capstone2-env/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


163/163 [==============================] - 2s 5ms/step - loss: 0.3762 - accuracy: 0.8340
Epoch 2/2
163/163 [==============================] - 0s 2ms/step
[CV 2/2] END mlp__model__num_layers=3, mlp__model__units=64;, score=0.907 total time=   7.0s
Epoch 1/2


/Users/ronlodetti/anaconda3/envs/capstone2-env/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


326/326 [==============================] - 2s 5ms/step - loss: 0.3278 - accuracy: 0.8627
Epoch 2/2
326/326 [==============================] - 1s 4ms/step - loss: 0.1634 - accuracy: 0.9391


In [24]:
# Balanced Accuracy Pipe
vectorizer = CountVectorizer(decode_error='replace', strip_accents='unicode', stop_words=None, ngram_range=(1, 2), max_df=0.95, min_df=2)
tf_idf = TfidfTransformer()
k_best = SelectKBest(k=10000)

pipe = Pipeline([
    ("count", vectorizer),
    ('tf_idf', tf_idf),
    ('feature_selection', k_best),
    ('mlp', model_wrapper)
])

params = {
    'mlp__model__num_layers': [2, 3],  
    'mlp__model__units': [32, 64]}

gs = GridSearchCV(estimator=pipe, 
                  param_grid=params,
                  scoring='balanced_accuracy',
                  cv=2,
                  verbose=3,
                  error_score='raise')

# The fitting process would be initiated with actual data
grid_search = gs.fit(X_train_clean, y_train)

Fitting 2 folds for each of 4 candidates, totalling 8 fits
Epoch 1/2


/Users/ronlodetti/anaconda3/envs/capstone2-env/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


163/163 [==============================] - 1s 3ms/step - loss: 0.5068 - accuracy: 0.8326
Epoch 2/2
163/163 [==============================] - 0s 2ms/step
[CV 1/2] END mlp__model__num_layers=2, mlp__model__units=32;, score=0.893 total time=   6.3s
Epoch 1/2


/Users/ronlodetti/anaconda3/envs/capstone2-env/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


163/163 [==============================] - 1s 4ms/step - loss: 0.5104 - accuracy: 0.8258
Epoch 2/2
163/163 [==============================] - 0s 2ms/step
[CV 2/2] END mlp__model__num_layers=2, mlp__model__units=32;, score=0.887 total time=   6.4s
Epoch 1/2


...

163/163 [==============================] - 2s 6ms/step - loss: 0.3654 - accuracy: 0.8404
Epoch 2/2
163/163 [==============================] - 0s 2ms/step
[CV 1/2] END mlp__model__num_layers=3, mlp__model__units=64;, score=0.895 total time=   7.5s
Epoch 1/2


/Users/ronlodetti/anaconda3/envs/capstone2-env/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


163/163 [==============================] - 2s 6ms/step - loss: 0.3762 - accuracy: 0.8340
Epoch 2/2
163/163 [==============================] - 0s 2ms/step
[CV 2/2] END mlp__model__num_layers=3, mlp__model__units=64;, score=0.900 total time=   6.9s
Epoch 1/2


/Users/ronlodetti/anaconda3/envs/capstone2-env/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


326/326 [==============================] - 2s 4ms/step - loss: 0.3278 - accuracy: 0.8627
Epoch 2/2
326/326 [==============================] - 1s 4ms/step - loss: 0.1634 - accuracy: 0.9391


In [25]:
# Adding EarlyStopping
def build_mlp_model(num_layers=2, units=64):
    model = Sequential()
    model.add(layers.InputLayer(input_shape=(10000,)))
    for _ in range(num_layers - 1):
        model.add(layers.Dense(units, activation="relu"))
        units = units // 2 
    model.add(layers.Dense(1, activation='sigmoid'))
    return model
    
CALLBACKS = [tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                              patience=5, 
                                              restore_best_weights=True,
                                              verbose=1,
                                              start_from_epoch=5)]

model_wrapper = KerasClassifier(build_fn=build_mlp_model,
                                random_state=42,
                                optimizer='adam',
                                loss='binary_crossentropy',
                                metrics=['accuracy'],
                                batch_size=64,
                                verbose=1,
                                callbacks=CALLBACKS,
                                shuffle=True,
                                epochs=2)
# Balanced Accuracy Pipe
vectorizer = CountVectorizer(decode_error='replace', strip_accents='unicode', stop_words=None, ngram_range=(1, 2), max_df=0.95, min_df=2)
tf_idf = TfidfTransformer()
k_best = SelectKBest(k=10000)

pipe = Pipeline([
    ("count", vectorizer),
    ('tf_idf', tf_idf),
    ('feature_selection', k_best),
    ('mlp', model_wrapper)
])

params = {
    'mlp__model__num_layers': [2, 3],  
    'mlp__model__units': [32, 64]}

gs = GridSearchCV(estimator=pipe, 
                  param_grid=params,
                  scoring='balanced_accuracy',
                  cv=2,
                  verbose=3,
                  error_score='raise')

# The fitting process would be initiated with actual data
grid_search = gs.fit(X_train_clean, y_train)

Fitting 2 folds for each of 4 candidates, totalling 8 fits
Epoch 1/2


/Users/ronlodetti/anaconda3/envs/capstone2-env/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


163/163 [==============================] - 2s 5ms/step - loss: 0.5068 - accuracy: 0.8326
Epoch 2/2
163/163 [==============================] - 0s 2ms/step
[CV 1/2] END mlp__model__num_layers=2, mlp__model__units=32;, score=0.893 total time=   6.4s
Epoch 1/2


/Users/ronlodetti/anaconda3/envs/capstone2-env/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


163/163 [==============================] - 1s 3ms/step - loss: 0.5104 - accuracy: 0.8258
Epoch 2/2
163/163 [==============================] - 0s 2ms/step
[CV 2/2] END mlp__model__num_layers=2, mlp__model__units=32;, score=0.887 total time=   5.7s
Epoch 1/2


...

163/163 [==============================] - 2s 6ms/step - loss: 0.3654 - accuracy: 0.8404
Epoch 2/2
163/163 [==============================] - 0s 2ms/step
[CV 1/2] END mlp__model__num_layers=3, mlp__model__units=64;, score=0.895 total time=   7.1s
Epoch 1/2


/Users/ronlodetti/anaconda3/envs/capstone2-env/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


163/163 [==============================] - 2s 6ms/step - loss: 0.3762 - accuracy: 0.8340
Epoch 2/2
163/163 [==============================] - 0s 2ms/step
[CV 2/2] END mlp__model__num_layers=3, mlp__model__units=64;, score=0.900 total time=   6.9s
Epoch 1/2


/Users/ronlodetti/anaconda3/envs/capstone2-env/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


326/326 [==============================] - 2s 4ms/step - loss: 0.3278 - accuracy: 0.8627
Epoch 2/2
326/326 [==============================] - 1s 4ms/step - loss: 0.1634 - accuracy: 0.9391


In [32]:
test = pd.DataFrame(grid_search.cv_results_)[['mean_fit_time', 'mean_score_time','param_mlp__model__num_layers', 'param_mlp__model__units','mean_test_score']]
test

,mean_fit_time,mean_score_time,param_mlp__model__num_layers,param_mlp__model__units,mean_test_score
0,4.286467,1.757064,2,32,0.890143
1,4.929097,1.776644,2,64,0.894142
2,4.251326,1.834930,3,32,0.899804
3,5.083485,1.921514,3,64,0.897508


In [35]:
test = pd.DataFrame(grid_search.cv_results_)[['mean_fit_time', 'mean_score_time','param_mlp__model__num_layers', 'param_mlp__model__units','mean_test_score']]
test['time'] = (test['mean_fit_time']+test['mean_score_time']).map(lambda x: int(x))
test['num_layers'] = test['param_mlp__model__num_layers']
test['units'] = test['param_mlp__model__units']
test['balanced_accuracy'] = test['mean_test_score']
final = test[['time','num_layers','units' ,'balanced_accuracy']]
final

,time,num_layers,units,balanced_accuracy
0,6,2,32,0.890143
1,6,2,64,0.894142
2,6,3,32,0.899804
3,7,3,64,0.897508


In [36]:
import pandas as pd

def summarize_grid_search_results(grid_search):
    columns_to_extract = [
        ('mean_fit_time', 'fit_time'),
        ('mean_score_time', 'score_time'),
        ('param_mlp__model__num_layers', 'num_layers'),
        ('param_mlp__model__units', 'units'),
        ('mean_test_score', 'balanced_accuracy')
    ]
    summary_df = pd.DataFrame(grid_search.cv_results_)[[original for original, renamed in columns_to_extract]]

    summary_df.columns = [renamed for original, renamed in columns_to_extract]
    
    # Calculate total time and convert to int
    summary_df['time'] = (summary_df['fit_time'] + summary_df['score_time']).astype(int)
    
    # Reorder and select final columns for the output
    final_columns = ['time', 'num_layers', 'units', 'balanced_accuracy']
    final_df = summary_df[final_columns]
    
    return final_df


In [38]:
def build_mlp_model(num_layers=2, units=64, initializer=None):
    model = Sequential()
    model.add(layers.InputLayer(input_shape=(20000,)))
    for _ in range(num_layers - 1):
        model.add(layers.Dense(units, activation="relu", kernel_initializer=initializer))
        units = units // 2 
    model.add(layers.Dense(1, activation='sigmoid'))
    return model
    
CALLBACKS = [tf.keras.callbacks.EarlyStopping(monitor='loss',
                                              min_delta=0.01,
                                              patience=5, 
                                              restore_best_weights=True,
                                              verbose=1,
                                              start_from_epoch=5)]

model_wrapper = KerasClassifier(build_fn=build_mlp_model,
                                random_state=42,
                                optimizer='adam',
                                loss='binary_crossentropy',
                                metrics=['accuracy'],
                                batch_size=64,
                                verbose=1,
                                callbacks=CALLBACKS,
                                shuffle=True,
                                epochs=20)

vectorizer = CountVectorizer(decode_error='replace', strip_accents='unicode', stop_words=None, ngram_range=(1, 2), max_df=0.95, min_df=2)
tf_idf = TfidfTransformer()
k_best = SelectKBest(k=20000)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

pipe = Pipeline([
    ("count", vectorizer),
    ('tf_idf', tf_idf),
    ('feature_selection', k_best),
    ('mlp', model_wrapper)
])

params = {
    'mlp__model__num_layers': [2, 3],  
    'mlp__model__units': [8, 16, 32, 64],
    'mlp__model__initializer': [None,'he_normal']}

gs = GridSearchCV(estimator=pipe, 
                  param_grid=params,
                  scoring='balanced_accuracy',
                  cv=skf,
                  verbose=3,
                  error_score='raise',
                  n_jobs= -1)

# The fitting process would be initiated with actual data
grid_search = gs.fit(X_train_clean, y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


2024-03-29 14:34:19.421920: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-29 14:34:19.422707: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-29 14:34:19.423300: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuil

Epoch 1/20


/Users/ronlodetti/anaconda3/envs/capstone2-env/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/ronlodetti/anaconda3/envs/capstone2-env/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/ronlodetti/anaconda3/envs/capstone2-env/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/ronlodetti/anaconda3/envs/capstone2-env/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, 

Epoch 1/20
Epoch 1/20
Epoch 1/20
Epoch 1/20
Epoch 1/20
Epoch 1/20
Epoch 1/20
261/261 [==============================] - 3s 11ms/step - loss: 0.2433 - accuracy: 0.9175
Epoch 3/20
261/261 [==============================] - 3s 11ms/step - loss: 0.1793 - accuracy: 0.9393
Epoch 4/20
261/261 [==============================] - 3s 10ms/step - loss: 0.1745 - accuracy: 0.9408
Epoch 5/20
261/261 [==============================] - 3s 10ms/step - loss: 0.1443 - accuracy: 0.9535
Epoch 6/20
261/261 [==============================] - 3s 11ms/step - loss: 0.1225 - accuracy: 0.9613
Epoch 7/20
261/261 [==============================] - 3s 11ms/step - loss: 0.1158 - accuracy: 0.9628
Epoch 6/20
Epoch 6/20
261/261 [==============================] - 3s 12ms/step - loss: 0.1065 - accuracy: 0.9682.........] - ETA: 3s - loss: 0.0880 - accuracy: 0.97
Epoch 8/20
261/261 [==============================] - 3s 12ms/step - loss: 0.0778 - accuracy: 0.9788............] - ETA: 1s - loss: 0.0889 - accuracy: 0.97
Epoch 8/

...

261/261 [==============================] - 7s 28ms/step - loss: 0.0489 - accuracy: 0.9857ss: 0.0023 - accuracy: 0.99
Epoch 4/20
210/261 [=======================>......] - ETA: 0s - loss: 8.5270e-04 - accuracy: 0.9999[CV 1/5] END mlp__model__initializer=he_normal, mlp__model__num_layers=3, mlp__model__units=32;, score=0.879 total time= 1.2min
Epoch 1/20
261/261 [==============================] - 4s 16ms/step - loss: 8.1648e-04 - accuracy: 0.9999
Epoch 11/20
148/261 [============================>.] - ETA: 0s - loss: 0.0175 - accuracy: 0.99

/Users/ronlodetti/anaconda3/envs/capstone2-env/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


261/261 [==============================] - 7s 25ms/step - loss: 0.0176 - accuracy: 0.9963
Epoch 5/20
261/261 [==============================] - 7s 25ms/step - loss: 0.0013 - accuracy: 0.9999
Epoch 8/20
261/261 [==============================] - 7s 26ms/step - loss: 0.0017 - accuracy: 1.0000
Epoch 7/20
261/261 [==============================] - 4s 15ms/step - loss: 5.8117e-04 - accuracy: 0.9999
Epoch 11: early stopping
232/261 [=========================>....] - ETA: 0s - loss: 0.0064 - accuracy: 0.9993[CV 2/5] END mlp__model__initializer=he_normal, mlp__model__num_layers=3, mlp__model__units=32;, score=0.860 total time= 1.2min
Epoch 1/20
261/261 [==============================] - 6s 22ms/step - loss: 0.0063 - accuracy: 0.9993
Epoch 6/20
261/261 [==============================] - 6s 21ms/step - loss: 0.0012 - accuracy: 0.9999
Epoch 9/20
261/261 [==============================] - 6s 22ms/step - loss: 8.3288e-04 - accuracy: 1.0000
Epoch 8/20
261/261 [==============================] - 7s 25

/Users/ronlodetti/anaconda3/envs/capstone2-env/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/20
326/326 [==============================] - 3s 4ms/step - loss: 0.5161 - accuracy: 0.8754
Epoch 2/20
326/326 [==============================] - 1s 4ms/step - loss: 0.2734 - accuracy: 0.9168
Epoch 3/20
326/326 [==============================] - 1s 4ms/step - loss: 0.1990 - accuracy: 0.9321
Epoch 4/20
326/326 [==============================] - 1s 4ms/step - loss: 0.1622 - accuracy: 0.9451
Epoch 5/20
326/326 [==============================] - 1s 3ms/step - loss: 0.1366 - accuracy: 0.9536
Epoch 6/20
326/326 [==============================] - 1s 3ms/step - loss: 0.1167 - accuracy: 0.9613
Epoch 7/20
326/326 [==============================] - 1s 3ms/step - loss: 0.0998 - accuracy: 0.9686
Epoch 8/20
326/326 [==============================] - 1s 3ms/step - loss: 0.0855 - accuracy: 0.9750
Epoch 9/20
326/326 [==============================] - 1s 3ms/step - loss: 0.0733 - accuracy: 0.9802
Epoch 10/20
326/326 [==============================] - 1s 3ms/step - loss: 0.0626 - accuracy: 0.9841

In [41]:
pd.DataFrame(grid_search.cv_results_).columns

Index(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_mlp__model__initializer', 'param_mlp__model__num_layers',
       'param_mlp__model__units', 'params', 'split0_test_score',
       'split1_test_score', 'split2_test_score', 'split3_test_score',
       'split4_test_score', 'mean_test_score', 'std_test_score',
       'rank_test_score'],
      dtype='object')

In [58]:
def summarize_grid_search_results(grid_search):
    columns_to_extract = [
        ('mean_fit_time', 'fit_time'),
        ('mean_score_time', 'score_time'),
        ('param_mlp__model__num_layers', 'num_layers'),
        ('param_mlp__model__units', 'units'),
        ('mean_test_score', 'balanced_accuracy'),
        ('param_mlp__model__initializer','initializer'),
        ('param_mlp__model__dropout_rate', 'dropout_rate')
        
    ]
    summary_df = pd.DataFrame(grid_search.cv_results_)[[original for original, renamed in columns_to_extract]]

    summary_df.columns = [renamed for original, renamed in columns_to_extract]
    
    # Calculate total time and convert to int
    summary_df['time'] = (summary_df['fit_time'] + summary_df['score_time']).astype(int)
    
    # Reorder and select final columns for the output
    final_columns = ['balanced_accuracy', 'time', 'num_layers', 'units', 'dropout_rate', 'initializer']
    final_df = summary_df[final_columns]
    sorted_df = final_df.sort_values(by=['balanced_accuracy', 'time'], ascending=[False, True])
    
    return sorted_df

In [52]:
summarize_grid_search_results(grid_search)

,balanced_accuracy,time,num_layers,units,initializer
8,0.878301,76,2,8,he_normal
0,0.877872,72,2,8,None
3,0.875127,145,2,64,None
2,0.873643,114,2,32,None
10,0.873609,115,2,32,he_normal
11,0.873384,162,2,64,he_normal
4,0.872424,153,3,8,None
6,0.871859,89,3,32,None
1,0.871578,90,2,16,None
9,0.871471,84,2,16,he_normal


In [54]:
# Need to add dropout layers
def build_mlp_model(num_layers=1, units=64, initializer=None, dropout_rate=0.2):
    model = Sequential()
    model.add(layers.InputLayer(input_shape=(20000,)))
    for _ in range(num_layers):
        model.add(layers.Dense(units, activation="relu", kernel_initializer=initializer))
        model.add(layers.Dropout(dropout_rate))
        units = units // 2 
    model.add(layers.Dense(1, activation='sigmoid'))
    return model
    
CALLBACKS = [tf.keras.callbacks.EarlyStopping(monitor='loss',
                                              min_delta=0.001,
                                              patience=5, 
                                              restore_best_weights=True,
                                              verbose=1,
                                              start_from_epoch=5)]

model_wrapper = KerasClassifier(model=build_mlp_model,
                                random_state=42,
                                optimizer='adam',
                                loss='binary_crossentropy',
                                metrics=['accuracy'],
                                batch_size=64,
                                verbose=0,
                                callbacks=CALLBACKS,
                                shuffle=True,
                                epochs=20)

vectorizer = CountVectorizer(decode_error='replace', strip_accents='unicode', stop_words=None, ngram_range=(1, 2), max_df=0.95, min_df=2)
tf_idf = TfidfTransformer()
k_best = SelectKBest(k=20000)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

pipe = Pipeline([
    ("count", vectorizer),
    ('tf_idf', tf_idf),
    ('feature_selection', k_best),
    ('mlp', model_wrapper)
])

params = {
    'mlp__model__num_layers': [2, 3],  
    'mlp__model__units': [8, 16, 32, 64],
    'mlp__model__initializer': [None,'he_normal'],
    'mlp__model__dropout_rate': [0.2, 0.3, 0.4, 0.5]}

gs = GridSearchCV(estimator=pipe, 
                  param_grid=params,
                  scoring='balanced_accuracy',
                  cv=skf,
                  verbose=1,
                  error_score='raise',
                  n_jobs= -1,
                  return_train_score=True)

# The fitting process would be initiated with actual data
grid_search = gs.fit(X_train_clean, y_train)

Fitting 5 folds for each of 64 candidates, totalling 320 fits


2024-03-29 21:10:31.123107: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-29 21:10:31.125311: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-29 21:10:31.125311: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuil

Restoring model weights from the end of the best epoch: 11.
Epoch 16: early stopping
Restoring model weights from the end of the best epoch: 13.
Epoch 18: early stopping
Restoring model weights from the end of the best epoch: 14.
Epoch 19: early stopping
Restoring model weights from the end of the best epoch: 15.
Epoch 20: early stopping
Restoring model weights from the end of the best epoch: 9.
Epoch 14: early stopping
Restoring model weights from the end of the best epoch: 9.
Epoch 14: early stopping
Restoring model weights from the end of the best epoch: 9.
Epoch 14: early stopping
Restoring model weights from the end of the best epoch: 9.
Epoch 14: early stopping
Restoring model weights from the end of the best epoch: 9.
Epoch 14: early stopping
Restoring model weights from the end of the best epoch: 8.
Epoch 13: early stopping
Restoring model weights from the end of the best epoch: 9.
Epoch 14: early stopping
Restoring model weights from the end of the best epoch: 11.
Epoch 16: ea

In [57]:
grid_search.cv_results_.keys()

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_mlp__model__dropout_rate', 'param_mlp__model__initializer', 'param_mlp__model__num_layers', 'param_mlp__model__units', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score', 'split0_train_score', 'split1_train_score', 'split2_train_score', 'split3_train_score', 'split4_train_score', 'mean_train_score', 'std_train_score'])

In [59]:
summarize_grid_search_results(grid_search)

,balanced_accuracy,time,num_layers,units,dropout_rate,initializer
36,0.894412,70,3,8,0.4,None
48,0.894191,92,2,8,0.5,None
53,0.892804,84,3,16,0.5,None
20,0.892495,88,3,8,0.3,None
52,0.891852,114,3,8,0.5,None
...,...,...,...,...,...,...
47,0.868468,181,3,64,0.4,he_normal
15,0.867966,129,3,64,0.2,he_normal
27,0.867665,183,2,64,0.3,he_normal
14,0.867195,103,3,32,0.2,he_normal
